In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import pickle

In [2]:
# backend
# loading single coin models
def coin_model_instance():
    window_size = 60
    model_coin = Sequential([
        LSTM(128, return_sequences=True, input_shape=(window_size, 1)),
        Dropout(0.2),
        LSTM(128, return_sequences=False),
        Dense(units=1)
    ])
    return model_coin


def reload_model(coin):
    window_size = 60
    model_paths = [
        './checkpoints/btc_checkpoint.weights.h5',
        './checkpoints/eth_checkpoint.weights.h5',
        './checkpoints/ltc_checkpoint.weights.h5'
    ]

    model_multicoin = Sequential(
        [LSTM(64, return_sequences=True, input_shape=(window_size, 5)),
        Dropout(0.2),
        LSTM(64, return_sequences=False),
        Dense(units=1)]
    )
    model_multicoin.load_weights('./checkpoints/merge_checkpoint.weights.h5')

    coin_id = -1
    if coin == 'BTC':
        coin_id = 0
    elif coin == "LTC":
        coin_id = 1
    elif coin == 'ETH':
        coin_id = 2

    coin_model = coin_model_instance()
    coin_model.load_weights(model_paths[coin_id])
    # print(model_multicoin.summary())
    # print(coin_model.summary())
    return model_multicoin, coin_model


def reload_scaler(coin):
    coin_paths = [
        './checkpoints/btc_scaler.pkl',
        './checkpoints/ltc_scaler.pkl',
        './checkpoints/eth_scaler.pkl',
    ]

    # multi-coin
    with open('./checkpoints/merge_scaler.pkl', 'rb') as f:
        merge_scaler = pickle.load(f)
        f.close()
    
    # single coin
    coin_id = -1
    if coin == 'BTC':
        coin_id = 0
    elif coin == "LTC":
        coin_id = 1
    elif coin == 'ETH':
        coin_id = 2
    
    with open(coin_paths[coin_id], 'rb') as f_coin:
        coin_scaler = pickle.load(f_coin)
        f_coin.close()

    return merge_scaler, coin_scaler

def fetch_current_data(coin='BTC'):
    btc = yf.Ticker('BTC-USD')
    ltc = yf.Ticker('LTC-USD')
    eth = yf.Ticker('ETH-USD')

    btc_hist = btc.history(period='60D')
    ltc_hist = ltc.history(period='60D')
    eth_hist = eth.history(period='60D')

    coin_list = [btc_hist, ltc_hist, eth_hist]
    for c in coin_list:
        if 'Stock Splits' in c.columns:
            c.drop(columns=['Stock Splits'], inplace=True)
        if 'Dividends' in c.columns:
            c.drop(columns=['Dividends'], inplace=True)

    '''
    For single coin, require only close price data.
    For multi-coin, require all 5 features
    '''

    btc_data = btc_hist
    eth_data = eth_hist
    ltc_data = ltc_hist

    if coin == "BTC":
        return btc_data
    elif coin == "ETH":
        return eth_data
    elif coin == 'LTC':
        return ltc_data


In [3]:
forecast_days=[1, 5, 10, 30]
coin='BTC'
target_col = 3

# Fetch initial inputs
X_merge_init = fetch_current_data(coin)  # shape (60, 5)
X_coin_init = X_merge_init['Close']      # assuming shape (60, 5) too

# Reload scalers and models
merg_scaler, coin_scaler = reload_scaler(coin)
merge_model, coin_model = reload_model(coin)

# Constants
total_features = X_merge_init.shape[-1]

# Convert to NumPy arrays
X_input_multicoin = X_merge_init.to_numpy()
X_input_coin = X_merge_init['Close'].to_numpy()

# Ensure shapes are correct
X_input_multicoin = X_input_multicoin.reshape(1, 60, total_features)
X_input_coin = X_input_coin.reshape(1, 60, 1)

# Store predictions
forecast_result_multicoin = {}
forecast_result_single = {}
predicted_closes_multicoin = []
predicted_closes_single = []

In [7]:
X_input_multicoin[0][0], X_input_multicoin.shape

(array([9.65778047e+04, 9.66718750e+04, 9.52704531e+04, 9.62739219e+04,
        1.69994790e+10]),
 (1, 60, 5))

In [ ]:
X_input_coin.shape

(1, 60, 1)

In [9]:
pred_scaled_multicoin = merge_model.predict(X_input_multicoin, verbose=0)
pred_scaled_coin = coin_model.predict(X_input_coin, verbose=0)

pred_scaled_multicoin, pred_scaled_coin

(array([[0.37435454]], dtype=float32), array([[8.808902]], dtype=float32))

In [10]:
pred_scaled_multicoin.flatten()

array([0.37435454], dtype=float32)

In [12]:
expanded_multicoin = np.zeros((1, total_features))
expanded_multicoin[:, target_col] = pred_scaled_multicoin.flatten()
expanded_multicoin

array([[0.        , 0.        , 0.        , 0.37435454, 0.        ]])

In [13]:
expanded_coin = np.zeros((1, total_features))
expanded_coin[:, target_col] = pred_scaled_coin.flatten()
expanded_coin

array([[0.        , 0.        , 0.        , 8.80890179, 0.        ]])

In [15]:
pred_full_multicoin = merg_scaler.inverse_transform(expanded_multicoin)
pred_full_coin = coin_scaler.inverse_transform(expanded_coin)

pred_full_multicoin, pred_full_coin

(array([[1.15323997e+00, 1.34481001e+00, 1.11373997e+00, 2.73598640e+04,
         4.81714000e+05]]),
 array([[1.78102997e+02, 1.78102997e+02, 1.78102997e+02, 5.93798764e+05,
         1.78102997e+02]]))

In [16]:
pred_close_multicoin = pred_full_multicoin[:, target_col][0]
pred_close_coin = pred_full_coin[:, target_col][0]

pred_close_multicoin, pred_close_coin

(np.float64(27359.86399725144), np.float64(593798.7641888537))

In [18]:
next_input_multi = X_input_multicoin[:, 1:, :].copy()
next_input_multi[0][0]

array([9.62779609e+04, 9.65034531e+04, 9.13717422e+04, 9.14181719e+04,
       4.40464805e+10])

In [19]:
new_step_multi = X_input_multicoin[:, -1, :].copy()
new_step_multi

array([[9.34661172e+04, 9.43918203e+04, 9.26785234e+04, 9.35913828e+04,
        5.99744512e+10]])

In [20]:
new_step_multi[0, target_col] = pred_scaled_multicoin.flatten()[0]
new_step_multi

array([[9.34661172e+04, 9.43918203e+04, 9.26785234e+04, 3.74354541e-01,
        5.99744512e+10]])

In [21]:
new_ = new_step_multi.reshape(1, 1, total_features)
new_

array([[[9.34661172e+04, 9.43918203e+04, 9.26785234e+04, 3.74354541e-01,
         5.99744512e+10]]])

In [22]:
X_input_multicoin = np.concatenate([next_input_multi, new_step_multi.reshape(1, 1, total_features)], axis=1)
X_input_multicoin

array([[[9.62779609e+04, 9.65034531e+04, 9.13717422e+04, 9.14181719e+04,
         4.40464805e+10],
        [9.14371172e+04, 9.25110781e+04, 8.60082344e+04, 8.87361719e+04,
         9.21391041e+10],
        [8.86388906e+04, 8.92862500e+04, 8.21318984e+04, 8.43470234e+04,
         6.45974921e+10],
        [8.40768594e+04, 8.70007812e+04, 8.31449609e+04, 8.47042266e+04,
         5.26595920e+10],
        [8.47056250e+04, 8.50363203e+04, 7.82489141e+04, 8.43730078e+04,
         8.36105706e+10],
        [8.43738672e+04, 8.65223047e+04, 8.37942344e+04, 8.60319141e+04,
         2.91906284e+10],
        [8.60362578e+04, 9.50434375e+04, 8.50402109e+04, 9.42483516e+04,
         5.83983411e+10],
        [9.42484219e+04, 9.44297500e+04, 8.50813047e+04, 8.60656719e+04,
         7.00722285e+10],
        [8.60640703e+04, 8.89112734e+04, 8.15292422e+04, 8.72221953e+04,
         6.80952415e+10],
        [8.72229531e+04, 9.09982422e+04, 8.63797734e+04, 9.06235625e+04,
         5.04989880e+10],
        [9